In [33]:
from sympy import *
import numpy as np 
init_printing()

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

D11,D12,D13,D21,D22,D23,D31,D32,D33 = symbols( 'D11,D12,D13,D21,D22,D23,D31,D32,D33' )


Dv = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
Ed = Matrix( [ [Exx], [Ezz], [Gxz] ] )
T  = Dv*Ed;
for i in range(3):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx + D12*Ezz + D13*Gxz;
Tzz = D21*Exx + D22*Ezz + D23*Gxz;
Txz = D31*Exx + D32*Ezz + D33*Gxz;


In [43]:
# Compressibility switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')

dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

GxzS  = 1/dz*(uC-uS)  + 1/dx*(vSE-vSW)
GxzN  = 1/dz*(uN-uC)  + 1/dx*(vNE-vNW) 
GxzSW = 1/dz*(uW-uSW) + 1/dx*(vSW-vSWW)
GxzNW = 1/dz*(uNW-uW) + 1/dx*(vNW-vNWW)
GxzSE = 1/dz*(uE-uSE) + 1/dx*(vSEE-vSE) 
GxzNE = 1/dz*(uNE-uE) + 1/dx*(vNEE-vNE)

ExxW  = 1/dx*(uC-uW)
ExxE  = 1/dx*(uE-uC)
ExxSW = 1/dx*(uS-uSW)
ExxSE = 1/dx*(uSE-uS)
ExxNW = 1/dx*(uN-uNW)
ExxNE = 1/dx*(uNE-uN)

EzzW  = 1/dz*(vNW-vSW);
EzzE  = 1/dz*(vNE-vSE);
EzzSW = 1/dz*(vSW-vSSW);
EzzSE = 1/dz*(vSE-vSSE);
EzzNW = 1/dz*(vNNW-vNW);
EzzNE = 1/dz*(vNNE-vNE);

divW  = ExxW + EzzW
divE  = ExxE + EzzE
divSW = ExxSW + EzzSW 
divSE = ExxSE + EzzSE
divNW = ExxNW + EzzNW 
divNE = ExxNE + EzzNE
divS  = 1/4*( divW + divE + divSW + divSE)
divN  = 1/4*( divW + divE + divNW + divNE)

#----------------------------------------#
ExxW = 1/dx*( uC- uW) - comp*1/3*divW
EzzW = 1/dz*(vNW-vSW) - comp*1/3*divW
GxzW = 1/4*(GxzS+GxzN+GxzSW+GxzNW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W'})
#----------------------------------------#
ExxE = 1/dx*( uE- uC) - comp*1/3*divE
EzzE = 1/dz*(vNE-vSE) - comp*1/3*divE
GxzE = 1/4*(GxzS+GxzN+GxzSE+GxzNE)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E'})
#----------------------------------------#
ExxS = 1/4*(ExxE+ExxW+ExxSE+ExxSW) - comp*1/3*divS
EzzS = 1/4*(EzzE+EzzW+EzzSE+EzzSW) - comp*1/3*divS
GxzS = 1/dz*(uC-uS) + 1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S'})
#----------------------------------------#
ExxN = 1/4*(ExxE+ExxW+ExxNE+ExxNW) - comp*1/3*divN
EzzN = 1/4*(EzzE+EzzW+EzzNE+EzzNW) - comp*1/3*divN
GxzN = 1/dz*(uN-uC) + 1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
Fx = -Fx
for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = -(D31N*(0.166666666666667*comp/dx - 0.25/dx) - D31S*(0.166666666666667*comp/dx - 0.25/dx) + 0.166666666666667*D32N*comp/dx - 0.166666666666667*D32S*comp/dx)/dz - (-D11W*((1.0L/3.0L)*comp/dx - 1/dx) - 1.0L/3.0L*D12W*comp/dx)/dx;
uC = -(-D33N/dz - D33S/dz)/dz - (D11E*((1.0L/3.0L)*comp/dx - 1/dx) - D11W*(-1.0L/3.0L*comp/dx + 1.0/dx) + (1.0L/3.0L)*D12E*comp/dx + (1.0L/3.0L)*D12W*comp/dx)/dx;
uE = -(D31N*(-0.166666666666667*comp/dx + 0.25/dx) - D31S*(-0.166666666666667*comp/dx + 0.25/dx) - 0.166666666666667*D32N*comp/dx + 0.166666666666667*D32S*comp/dx)/dz - (D11E*(-1.0L/3.0L*comp/dx + 1.0/dx) - 1.0L/3.0L*D12E*comp/dx)/dx;
uS = -D33S/pow(dz, 2) - (-0.25*D13E/dz + 0.25*D13W/dz)/dx;
uN = -D33N/pow(dz, 2) - (0.25*D13E/dz - 0.25*D13W/dz)/dx;
vSW = -(0.166666666666667*D31N*comp/dz - 0.0833333333333333*D31S*comp/dz + D32N*(0.166666666666667*comp/dz - 0.25/dz) - 0.0833333333333333*D32S*comp/dz + D33S/dx)/dz - (-1.0L/3.0L*D11W*comp/dz - D12W*((1.0L/3.0L)*comp/dz - 1/dz) - 0.25*D13E/dx)/dx;
vSE

In [44]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')

dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

GxzSW = 1/dz*(uSW-uSSW) + 1/dx*(vS-vSW)
GxzSE = 1/dz*(uSSE-uSE) + 1/dx*(vSE-vS)
GxzW  = 1/dz*(uNW-uSW)  + 1/dx*(vC-vW)
GxzE  = 1/dz*(uNE-uSE)  + 1/dx*(vE-vC)
GxzNW = 1/dz*(uNW-uNNW) + 1/dx*(vN-vNW)
GxzNE = 1/dz*(uNNE-uNE) + 1/dx*(vNE-vN)

EzzS  = 1/dz*(vC-vS);
EzzN  = 1/dz*(vN-vC);
EzzSW = 1/dz*(vW-vSW);
EzzNW = 1/dz*(vNW-vW);
EzzSE = 1/dz*(vE-vSE);
EzzNE = 1/dz*(vNE-vE)

ExxS  = 1/dx*(uSE-uSW);
ExxN  = 1/dx*(uNE-uNW);
ExxSW = 1/dx*(uSW-uSWW);
ExxNW = 1/dx*(uNW-uNWW);
ExxSE = 1/dx*(uSEE-uSE);
ExxNE = 1/dx*(uNEE-uNE);

divS  = ExxS + EzzN
divN  = ExxN + EzzN
divSW = ExxSW + EzzSW;
divNW = ExxNW + EzzNW;
divSE = ExxSE + EzzSE;
divNE = ExxNE + EzzNE;
divW  = 1/4*(divS + divN + divSW + divNW) 
divE  = 1/4*(divS + divN + divSE + divNE)

#----------------------------------------#
ExxS = 1/dx*(uSE-uSW) - comp*1/3*divS
EzzS = 1/dz*( vC- vS) - comp*1/3*divS
GxzS = 1/4*(GxzW+GxzE+GxzSW+GxzSE)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S'})
#----------------------------------------#
ExxN = 1/dx*(uNE-uNW) - comp*1/3*divN
EzzN = 1/dz*( vN- vC) - comp*1/3*divN
GxzN = 1/4*(GxzW+GxzE+GxzNW+GxzNE)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N'})
#----------------------------------------#
ExxW = 1/4*(ExxS+ExxN+ExxSW+ExxNW) - comp*1/3*divW
EzzW = 1/4*(EzzS+EzzN+EzzSW+EzzNW) - comp*1/3*divW
GxzW = 1/dz*(uNW-uSW) + 1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W'})
# #----------------------------------------#
ExxE = 1/4*(ExxS+ExxN+ExxSE+ExxNE) - comp*1/3*divE
EzzE = 1/4*(EzzS+EzzN+EzzSE+EzzNE) - comp*1/3*divE
GxzE = 1/dz*(uNE-uSE) + 1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E'})
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = -D33W/pow(dx, 2) - (-0.25*D23N/dx + 0.25*D23S/dx)/dz;
vC = -((1.0L/3.0L)*D21N*comp/dz - 1.0L/3.0L*D21S*comp/dz + D22N*((1.0L/3.0L)*comp/dz - 1/dz) - D22S*((1.0L/3.0L)*comp/dz + 1.0/dz))/dz - (0.333333333333333*D31E*comp/dz - 0.333333333333333*D31W*comp/dz + 0.333333333333333*D32E*comp/dz - 0.333333333333333*D32W*comp/dz - D33E/dx - D33W/dx)/dx;
vE = -D33E/pow(dx, 2) - (0.25*D23N/dx - 0.25*D23S/dx)/dz;
vS = -D22S/pow(dz, 2) - (-0.25*D32E/dz + 0.25*D32W/dz)/dx;
vN = -(-1.0L/3.0L*D21N*comp/dz + (1.0L/3.0L)*D21S*comp/dz + D22N*(-1.0L/3.0L*comp/dz + 1.0/dz) + (1.0L/3.0L)*D22S*comp/dz)/dz - (-0.333333333333333*D31E*comp/dz + 0.333333333333333*D31W*comp/dz + D32E*(-0.333333333333333*comp/dz + 0.25/dz) - D32W*(-0.333333333333333*comp/dz + 0.25/dz))/dx;
uSW = -(-D21S*((1.0L/3.0L)*comp/dx - 1/dx) - 1.0L/3.0L*D22S*comp/dx - 0.25*D23N/dz)/dz - (D31E*(0.166666666666667*comp/dx - 0.25/dx) - 0.0833333333333333*D31W*comp/dx + 0.166666666666667*D32E*comp/dx - 0.0833333333333333*D32W*comp/dx + D33W